# PREDICTION WEB APPLICATION

### Importing liabriess 

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sb
import os
import glob
import json 
import requests
from bs4 import BeautifulSoup
%matplotlib inline

#### Creating function for Downloading Data For Training and Testing The Model

In [2]:
# os.listdir('./Prediction_train&test_Data')
def load_data(url):
    
    folderName = './Prediction_train&test_Data/'
    fileName = 'league_data.json'
    path = os.path.join(folderName, fileName)

    if os.path.exists(path):
        files = glob.glob('./Prediction_train&test_Data/*')
        for f in files:
            os.remove(f)

    response = requests.get(url)
    # print(path)

    with open(path, 'wb') as outfile:
        outfile.write(response.content)
     

        
url = 'https://vsmobile.bet9ja.com/shopadmin/standings_view/league_data.php?pid=14001'
load_data(url)

#### Reading Json Files 

In [69]:
folderName = './Prediction_train&test_Data/'
fileName = 'league_data.json'
path = os.path.join(folderName, fileName)

with open(path, 'r') as outfile:
    json_obj = json.load(outfile)

In [70]:
clubes = json_obj['teams']
matches = json_obj['matches']
teams = []
resulte = []
wonmarke = []

count = 0
for key, val in matches.items():
    
    teams.append({key: []})
    resulte.append({key: []})
    wonmarke.append({key: []})
    
    for x in val:
        
        teams[count][key].append([val[x]['teamA'], val[x]['teamB']])
        resulte[count][key].append(val[x]['result'])
        wonmarke[count][key].append(val[x]['wonmarkets'])
        
    count +=1

In [71]:
# json_obj
# teams
# resulte
# wonmarke

#### Wranggling Data into Data Base

- Data
    - columns:
        - teams
        - winners(0 home, 1 draw, 2 away)



In [72]:
# resulte

In [73]:
# wonmarke
# teams

In [74]:
# events = []
events = {}
teams_index = {}
for x in range(len(teams)):
    for y, obj in teams[x].items():
        events[y] = []
        teams_index[y] = []
        for i in obj:
            teamA = clubes[i[0]]['fifa']
            teamB = clubes[i[1]]['fifa']
            events[y].append(f'{teamA}-{teamB}')
            teams_index[y].append('-'.join(i))

In [76]:
score = {}
for x in range(len(wonmarke)):
    for y, string in wonmarke[x].items():
        score[y] = []
        for j in string:
            resulte_win = j.split(',')[0]
            score[y].append(resulte_win)

In [88]:
# score
# events
socore_dataframe = pd.DataFrame(score)
events_dataframe = pd.DataFrame(events)
teams_index_dataframe = pd.DataFrame(teams_index)

In [78]:
socore_dataframe.head()

,1,2,3,4,5,6,7,8,9,10,11
0,Home,Away,Draw,Home,Away,Home,Away,Home,Draw,Home,Away
1,Away,Home,Away,Away,Away,Home,Home,Home,Home,Draw,Home
2,Home,Draw,Away,Away,Home,Away,Draw,Home,Away,Away,Home
3,Home,Draw,Home,Home,Draw,Home,Home,Home,Home,Away,Draw
4,Home,Away,Home,Away,Home,Away,Draw,Draw,Draw,Home,Draw


In [91]:
events_dataframe.head()

,1,2,3,4,5,6,7,8,9,10,11,index
0,LIV-WHU,ARS-BUR,EVE-WHU,WHU-BUR,EVE-LEI,LEE-WAT,WAT-WHU,CRY-MNC,EVE-MNU,TOT-EVE,BUR-TOT,1
1,MNC-CHE,LEE-SOU,SOU-ARS,LEE-ARS,BUR-CHE,LEI-BUR,EVE-BRN,BRN-BUR,LIV-CRY,BRI-LIV,NWC-BRI,1
2,MNU-LEE,CHE-LIV,NWC-CHE,BRN-MNC,WOL-MNU,BRN-NWC,NWC-ASV,CHE-WAT,ARS-NOR,MNU-BUR,WHU-NOR,1
3,BRI-NOR,ASV-CRY,BRI-ASV,CHE-EVE,MNC-ASV,CHE-SOU,SOU-LEI,ASV-EVE,MNC-BRI,LEE-CHE,CHE-LEI,1
4,NWC-WAT,WAT-EVE,WOL-BRN,LEI-NWC,CRY-LEE,ASV-LIV,ARS-CHE,NOR-SOU,NWC-TOT,NOR-WAT,MNC-LEE,1


In [92]:
teams_index_dataframe.head()

,1,2,3,4,5,6,7,8,9,10,11
0,45-58,48-64,49-58,58-64,49-63,827-70,70-58,55-46,49-50,51-49,64-51
1,46-47,827-52,52-48,827-48,64-47,63-64,49-972,972-64,45-55,71-45,53-71
2,50-827,47-45,53-47,972-46,525-50,972-53,53-60,47-70,48-66,50-64,58-66
3,71-66,60-55,71-60,47-49,46-60,47-52,52-63,60-49,46-71,827-47,47-63
4,53-70,70-49,525-972,63-53,55-827,60-45,48-47,66-52,53-51,66-70,46-827


In [80]:
def melt_table(dataframe):
    dataframe['index'] = 1
    values_vars = [str(x) for x in range(1, dataframe.shape[1])]
    resulte = pd.melt(dataframe, id_vars=['index'], value_vars=values_vars)
    return resulte

In [93]:
new_events_df = melt_table(events_dataframe).rename(columns={'value': 'events', 'variable': 'weeks'})[['events','weeks']]
new_score_df = melt_table(socore_dataframe).rename(columns={'value': 'score', 'variable': 'weeks'})[['score','weeks']]
teams_index_df = melt_table(teams_index_dataframe).rename(columns={'value': 'teams_index', 'variable': 'weeks'})[['teams_index','weeks']]

In [82]:
new_events_df.head()

,events,weeks
0,LIV-WHU,1
1,MNC-CHE,1
2,MNU-LEE,1
3,BRI-NOR,1
4,NWC-WAT,1


In [83]:
new_score_df.head()

,score,weeks
0,Home,1
1,Away,1
2,Home,1
3,Home,1
4,Home,1


In [94]:
teams_index_df.head()

,teams_index,weeks
0,45-58,1
1,46-47,1
2,50-827,1
3,71-66,1
4,53-70,1


In [102]:
events_score_df = pd.concat([new_events_df, new_score_df, teams_index_df], axis=1, join="inner").iloc[:, np.r_[0:2, 2, 4]] # [['events', 'score', 'teams_index']]

In [103]:
events_score_df.head(6)

,events,weeks,score,teams_index
0,LIV-WHU,1,Home,45-58
1,MNC-CHE,1,Away,46-47
2,MNU-LEE,1,Home,50-827
3,BRI-NOR,1,Home,71-66
4,NWC-WAT,1,Home,53-70
5,TOT-ASV,1,Away,51-60


#### Saving Data Into CSV file

In [97]:
events_score_df.to_csv('./Prediction_train&test_Data/premier_leage_data.csv', index=False)